In [1]:
#spectrograms made with STFT
#not used in the final analysis

In [ ]:
import sys
import matplotlib.pyplot as plt
import datajoint as dj
import neurodsp.filt
import scipy
import numpy as np
import pandas as pd
from pathlib import Path

from statsmodels.stats import multitest
from numpy.random import RandomState
from permute.core import one_sample, two_sample
from netneurotools import stats
import mne
import matplotlib.gridspec as gridspec

import utils.helper_func as hf
import utils.plotting as plotting

In [4]:
top_dir = '/home/anastasia/epiphyte/anastasia/output'
folder = '03-spectrogram_FFT'

patient_id = 53
session_nr = 1

df_patient_info = pd.read_csv(f'{top_dir}/{patient_id}_channel_info.csv')

fs = 32768
#The resulting sample rate is up / down times the original sample rate.
up = 1
down=32
fs_downs = (up/down)*fs
dt = 1/fs_downs      # sampling period/time/interval or time resolution, often denoted as T

df_stim_info = pd.read_csv(f'{top_dir}/{patient_id}_df_stim_info.csv')
df_stim_info = df_stim_info.loc[df_stim_info['patient_id'] == patient_id]
#df_stim_info = df_stim_info.reset_index()
df_stim_info = df_stim_info.replace('???', 'unknown')
df_stim_info.to_csv(f'{top_dir}/{patient_id}_df_stim_info.csv')

all_stim = np.unique(df_stim_info['stim_id'])
all_stim_name = np.unique(df_stim_info['stim_name'])
all_brain_areas = np.unique(df_patient_info['recording_site'])

alpha = 0.001
prng = RandomState(50)
n_permutations = 5000
method = 'simes-hochberg'
test_type = 'perm_rel'

columns = ['channel_name', 'channel_location', 'stim_index', 'stim_name', 'paradigm', 'corr_results', 'locations1', 'locations2']

nperseg = 50 #window length
nfft = 100 #how many points for the FFT
noverlap = 40 #the number of overlapping samples between windows
signal_len = np.int(fs_downs*1.5)+1

#size of future spectrograms for preallocation
freq_bins = nfft/2+1
ncol = np.fix((signal_len-noverlap)/(nperseg-noverlap))
estim = (np.int(freq_bins), np.int(ncol))

#time points for the baseline
#t1 = -100
t1 = -500
t2 = -50

times = np.linspace(-0.5, 1, signal_len)
times = times*1000
time_zero_idx = np.where(times == hf.find_nearest(times, 0))[0][0]

In [3]:
#check if save directories exist
parent_dir = f'{top_dir}/{folder}/spectrograms/raw'
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)
    
parent_dir = f'{top_dir}/{folder}/spectrograms/log'
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)

parent_dir = f'{top_dir}/{folder}/spectrograms/normalized/db'
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)

parent_dir = f'{top_dir}/{folder}/spectrograms/normalized/zscore'
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)

parent_dir = f'{top_dir}/{folder}/plots'
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)

parent_dir = f'{top_dir}/{folder}/plots/db'
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)
    
parent_dir = f'{top_dir}/{folder}/plots/zscore'
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)
    
parent_dir = f'{top_dir}/{folder}/plots/db'  
hf.create_folder_structure(parent_dir, df_patient_info)

parent_dir = f'{top_dir}/{folder}/plots/zscore'  
hf.create_folder_structure(parent_dir, df_patient_info)

FileExistsError: [Errno 17] File exists: '/home/anastasia/epiphyte/anastasia/output/03-spectrogram_FFT/plots/db/CSC1_LA1'

In [6]:
#compute spectrogram
for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    epochs = np.load(f'{top_dir}/01-preprocessed/epochs/broadband_2_sec/{patient_id}_epochs_channel-{ch}.npy')
    
    for st in all_stim:
        
        pre_sp_array, post_sp_array, freq, time_sp, current_stim_index, current_stim_name = hf.compute_spectrogram(
                                        df_stim_info, st, estim, epochs, fs_downs, nperseg, nfft, noverlap)
        
        np.save(f'{top_dir}/{folder}/spectrograms/raw/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_pre.npy', pre_sp_array)
        np.save(f'{top_dir}/{folder}/spectrograms/raw/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_post.npy', post_sp_array)

        pre_sp_array_log = np.log10(pre_sp_array)
        post_sp_array_log = np.log10(post_sp_array)
        np.save(f'{top_dir}/{folder}/spectrograms/log/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_pre.npy', pre_sp_array_log)
        np.save(f'{top_dir}/{folder}/spectrograms/log/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_post.npy', post_sp_array_log)

t = np.linspace(t1, 1000, time_sp.shape[0])
np.save(f'{top_dir}/{folder}/spectrograms/freq.npy', freq)
np.save(f'{top_dir}/{folder}/spectrograms/times.npy', t)

In [8]:
#normalize
t = np.load(f'{top_dir}/{folder}/spectrograms/times.npy')
sp_dir = f'{top_dir}/{folder}/spectrograms'

for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    
    baseline_pre, baseline_post = hf.compute_baseline_per_channel(ch, ch_site, df_stim_info, sp_dir, estim, all_stim, t, t1, t2, sp_type='raw')
    baseline_pre_log, baseline_post_log = hf.compute_baseline_per_channel(ch, ch_site, df_stim_info, sp_dir, estim, all_stim, t, t1, t2, sp_type='log')
    
    for st in all_stim:
        pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
        pre = pre.reset_index(drop=True)
        post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
        
        current_stim_index = pre.loc[0,'stim_id']
        current_stim_name = pre.loc[0,'stim_name']
        current_stim_paradigm = pre.loc[0,'paradigm']
        
        pre_sp_array = np.load(f'{top_dir}/{folder}/spectrograms/raw/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_pre.npy')
        post_sp_array = np.load(f'{top_dir}/{folder}/spectrograms/raw/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_post.npy')

        #baseline normalization & dB
        pre_sp_array_db = hf.db_normalize(pre_sp_array, baseline_pre)
        post_sp_array_db = hf.db_normalize(post_sp_array, baseline_post)
        np.save(f'{top_dir}/{folder}/spectrograms/normalized/db/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_pre_db.npy', pre_sp_array_db)
        np.save(f'{top_dir}/{folder}/spectrograms/normalized/db/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_post_db.npy', post_sp_array_db)
        
        #baseline normalization & z-score
        pre_sp_array_log = np.load(f'{top_dir}/{folder}/spectrograms/log/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_pre.npy')
        post_sp_array_log = np.load(f'{top_dir}/{folder}/spectrograms/log/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_post.npy')

        pre_sp_array_zscore = hf.baseline_zscore(pre_sp_array_log, baseline_pre_log)
        post_sp_array_zscore = hf.baseline_zscore(post_sp_array_log, baseline_post_log)
        
        np.save(f'{top_dir}/{folder}/spectrograms/normalized/zscore/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_pre_zscore.npy', pre_sp_array_zscore)
        np.save(f'{top_dir}/{folder}/spectrograms/normalized/zscore/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_post_zscore.npy', post_sp_array_zscore)
        

In [3]:
#create organized plots
freq = np.load(f'{top_dir}/{folder}/spectrograms/freq.npy')
t = np.load(f'{top_dir}/{folder}/spectrograms/times.npy')
#we will work only with frequencies below 200
idx = np.where(freq == hf.find_nearest(freq, 200))[0][0]
freq = freq[0:idx+1]

for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    
    norm_type = 'db'
    plotting.plot_organized_spectrograms(df_stim_info, all_stim_name, t, freq, idx, ch, ch_site, folder, norm_type, alpha, method, test_type)
    
    norm_type = 'zscore'
    plotting.plot_organized_spectrograms(df_stim_info, all_stim_name, t, freq, idx, ch, ch_site, folder, norm_type, alpha, method, test_type)
    
    

Permuting 9999 times...


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    1.0s remaining:    1.0s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    1.1s finished
/home/hitchhiker/Documents/epiphyte/anastasia/code/utils/plotting.py:230: UserWarning: No contour levels were found within the data range.
  ax2.contour(t, freq, pval_corrected<=alpha, 0, colors='black')
/home/hitchhiker/Documents/epiphyte/anastasia/code/utils/plotting.py:235: UserWarning: No contour levels were found within the data range.
  ax3.contour(t, freq, pval_corrected<=alpha, 0, colors='black')


Permuting 9999 times...


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    0.2s finished


Permuting 9999 times...


...

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    0.3s finished


Permuting 9999 times...


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    0.3s finished


Permuting 9999 times...


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    0.3s finished


In [ ]:
#create plots
freq = np.load(f'{top_dir}/{folder}/spectrograms/freq.npy')
t = np.load(f'{top_dir}/{folder}/spectrograms/times.npy')
#we will work only with frequencies below 200
idx = np.where(freq == hf.find_nearest(freq, 200))[0][0]
freq = freq[0:idx+1]

for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    
    norm_type = 'db'
    plotting.plot_all_spectrograms_separately(df_stim_info, all_stim_name, t, freq, idx, ch, ch_site, folder, norm_type, alpha, method)
    
    norm_type = 'zscore'
    plotting.plot_all_spectrograms_separately(df_stim_info, all_stim_name, t, freq, idx, ch, ch_site, folder, norm_type, alpha, method)
    